### Importing libraries:

In [1]:
import pickle
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as ss
from pandas.plotting import scatter_matrix
import seaborn as sns
import pandas.testing as pdt
import scipy.stats as ss
import numpy.testing as npt
import pandas.testing as pdt

sns.set_style("darkgrid")

# <b>Title : Investigating factors that determine the property value in Mbombela </b>

### Compiled by
1. Bongumusa Mavuso(1682836)
2. Siyabonga Hlomuka (1384685)
3. Thobelani Makeleni (1199116)

# <b>Executive Summary:</b>

The purpose of this notebook is to investigate factors that influence the property value of the proparties of the Mbombela Municipality.

The notebook will model the relationship between different these factors and the property value using the dataset of from the property evaluation roll that was conducted in 2010 within the area of Mbombela by the Mbombela Municipality.

#### Limitations:
The limitations of the dataset are that this kind of study has not being investigated before, and the data available does not provide much information on how the properties prices are evaluated.

# Introduction

Every four years, the local municipalities in South Africa conduct a general evaluation roll that determines the municipal values of homes and the property rates that should be paid based on these values and ultimately, the price of the property. The property owners usually have objections to the property values determined by the municipality as these values are usually too low for a property value or too high for the prices that the owners must pay for using the municipality services. The property owners usually do not know how the municipality got to the property prices that come after the evaluation, as with the recent evaluation in where property owners experienced a hike in municipality bills. \[ [Private Property](https://www.privateproperty.co.za/advice/news/articles/property-valuations-shock-what-homeowners-can-do/6230)\]

In this project, we are investigating the factors that contribute to the property evalution prices using the dataset from Mbombela Municipality Evaluation Roll for 2010. This kind of analysis has not been done before, 
Useful sites to extend this introduction

## Question
 ### What factors dertermine the property value and to what extend do these factors contribute to the value of the property?
 
\*********
 
 
1. [Schindlers](https://www.schindlers.co.za/2017/howdovalrollswork/)


## Overview of methology
\*********

## Section Contents overview
1. [Data Description](#Data-Description)


# <b> Data Description: </b>

The dataset was obtained from [Open Africa:  Mbombela Municipality Property Valuation 2010 ](https://africaopendata.org/dataset/mbombela-municipality-property-valuation-2010) and downloaded on the 18 April 2021. The dataset contains over 57000 data points which have 49 features that describe some of the properties in Mbombela Municipality. 

#### Date collected
The data was collected in Mbombela Municipal Council, in Mpumalanga, South Africa, in 2010. The last time the data modified was on December 21, 2017, 1:09 PM (UTC+02:00)
 
#### Dataset before cleaning
The original dataset consists of 49 columns, 48 showing different factors contributing to the price of property in Mbombela. The description of these columns is given below:

- Township = The township in Mbombeni.
- Stand No = Stand number is used to identidy the property
- Sect Title No = Section Title Number. Sectional Title, as a form of ownership (as per the Sectional Titles Act No.95 of 1986), emerged originally to permit parties to buy a piece or section of a larger property / building / development in a fashion where there ownership (or title) is protected (under Sectional Title law) and where there are clear rules and guidelines on how the overall property is managed, maintained and run. 
- StandDescrip = Stand Description.
- Owner Name = The owner of the property.
- Account No = Account Number. Account Number of the owner.
- Prop Category = Property Category
- TariffCode = Tariff Code.
- Physical Addr = Physical Address Showing the physical location of the property.
- Street No = Street Number. Identifying the street where the property is located
- Size = The size of the property measured in square meters
- Up-Level Size
- MarketValue = The price which the property is valued at. (Our Predictive Variable)
- Exemp Size
- Exemp Value
- Main Roll No
- Supp RollNo = Supplimentary roll number.Every property in every municipality should (hypothetically) be on a roll, but because properties are continuously coming into existence and ceasing to exist, new rolls are created (these are referred to as supplementary rolls) to include any properties that have not been previously recorded on another, prior, general roll.
- Owner Type
- Valuation Date
- Effect Date
- Up-LevelInd
- Zone Code
- Zone Descrip
- Usage Code
- UsageDescrip
- Empty Stand Ind
- Valuation Type
- SGNumber
- Ward
- Cycle
- Meter Book
- Status
- Val Cnt
- Postal Addr1
- Postal Addr2
- Postal Addr3
- Postal Addr4
- Postal Addr5
- Postal Code
- BC Create Date
- BC Orig Date
- BC Altern Date
- BC Value
- BC Tariff
- Deed Date
- Deed No
- Register Date
- Register No
- Capture Date

#### Aspects of Data Quality 
- The data was retrieved from openAFRICA , openAFRICA is not a government portal. Instead, it’s a grassroots initiative, maintained by Code for Africa, as a public service.OpenAFRICA has additional support from Amazon Web Services (AWS) and the World Bank, both well known and accredited organisation thus making our source openAFRICA an accredible source.

- The data collection was done in 2010 and the evaluations of Market Value where done around the same time interval 2008 to 2010 making our Market values comparison fair because the amounts where not effected by other time factors like infaltion. 

- The data was collected from the same region(Mbombeni)

<h3>Reading in data:</h3>

In [2]:
data = pd.read_excel('valuation-2010.xls')

### Validations

In [46]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57252 entries, 0 to 57251
Data columns (total 49 columns):
 #   Column                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [3]:
data.describe()

,Account No,Size,Up-Level Size,MarketValue,Exemp Size,Exemp Value,Main Roll No,Supp RollNo,Valuation Date,Effect Date,...,Val Cnt,Postal Code,BC Create Date,BC Orig Date,BC Altern Date,BC Value,BC Tariff,Deed Date,Register Date,Capture Date
count,5.723900e+04,5.724600e+04,57246.0,5.724600e+04,57246.0,57246.0,57246.0,57246.000000,5.724600e+04,5.724600e+04,...,57246.000000,57227.000000,57227.0,57227.0,57227.0,0.0,0.0,5.722700e+04,5.722700e+04,5.722700e+04
mean,2.632920e+06,4.980989e+04,0.0,6.644221e+05,0.0,0.0,2.0,1.442913,2.009117e+07,2.009117e+07,...,1.000943,1900.546980,0.0,0.0,0.0,NaN,NaN,2.202494e+06,9.767406e+06,2.877971e+07
std,9.302737e+05,8.205704e+05,0.0,3.786126e+06,0.0,0.0,0.0,1.055363,2.034716e+03,2.034717e+03,...,0.040940,2221.613457,0.0,0.0,0.0,NaN,NaN,6.263835e+06,1.001051e+07,4.032805e+07
min,1.000855e+06,0.000000e+00,0.0,0.000000e+00,0.0,0.0,2.0,1.000000,2.009070e+07,2.009070e+07,...,1.000000,0.000000,0.0,0.0,0.0,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.114612e+06,3.520000e+02,0.0,5.600000e+04,0.0,0.0,2.0,1.000000,2.009070e+07,2.009070e+07,...,1.000000,1200.000000,0.0,0.0,0.0,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.506226e+06,5.420000e+02,0.0,2.400000e+05,0.0,0.0,2.0,1.000000,2.009070e+07,2.009070e+07,...,1.000000,1200.000000,0.0,0.0,0.0,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00
75%,3.062104e+06,1.108000e+03,0.0,7.000000e+05,0.0,0.0,2.0,1.000000,2.009070e+07,2.009070e+07,...,1.000000,1240.000000,0.0,0.0,0.0,NaN,NaN,0.000000e+00,2.004071e+07,2.009092e+07
max,1.564422e+07,9.327444e+07,0.0,6.600000e+08,0.0,0.0,2.0,6.000000,2.010012e+07,2.010012e+07,...,4.000000,9999.000000,0.0,0.0,0.0,NaN,NaN,2.009082e+07,2.202093e+07,1.000000e+08


In [4]:
data.head()

,Township,Stand No,Sect Title No,StandDescrip,Owner Name,Account No,Prop Category,TariffCode,Physical Addr,Street No,...,BC Create Date,BC Orig Date,BC Altern Date,BC Value,BC Tariff,Deed Date,Deed No,Register Date,Register No,Capture Date
0,NELSPRUIT TOWN,000000000000010000100000000,NaN,GED 1 ERF 1,NELSPRUIT HARDWARE,2218141.0,BUS,VABUS,PAUL KRUGER STREET,1,...,0.0,0.0,0.0,NaN,NaN,0.0,NaN,19810925.0,T46343/81,99999999.0
1,NELSPRUIT TOWN,00000000000001000RE00000000,NaN,RESTANT VAN ERF 1,MINTY IE & SONS PTY LTD,2218119.0,BUS,VABUS,ANDERSON STREET,36,...,0.0,0.0,0.0,NaN,NaN,0.0,NaN,19810925.0,T46344/81,99999999.0
2,NELSPRUIT TOWN,000000000000020000000000000,NaN,ERF 2,MINTYS UNIVERSAL STORES LTD,2218130.0,BUS,VABUS,ANDREW STREET,33,...,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,T96299/200,0.0
3,NELSPRUIT TOWN,000000000000030000000000000,NaN,ERF 3,LOWMIN PROPERTIES (PTY) LTD,2217568.0,BUS,VABUS,ANDERSON STREET,32,...,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,T96299/200,0.0
4,NELSPRUIT TOWN,000000000000040000000000000,NaN,ERF 4,NELMIN ERF 4 PROPERTIES (PTY) LTD,2040436.0,BUS,VABUS,ANDERSON STREET,30,...,0.0,0.0,0.0,NaN,NaN,0.0,NaN,20020814.0,T96299/200,20021029.0


<h1><b>Data processing / Exploratory data analysis:</b></h1>

<blockqoute>
In this next cell we drop several columns of the dataset either because we think they were irrelevent in determining the value of the property or the data had a lot of missing values or the information contained in the column was redundant as it was contained in another column.
</blockqoute>

In [5]:
#Columns to be removed
#Stand No, Sect Tittle, Account Number, TariffCode, Street No, Up-Level Size,
#Exemp Size, Main Roll No, Owner Type, Valuation Date, Up-levelInd, Usage Description
#Zone Description, SG Number, Status, Postal Address(1,2, 3, 4, 5), All columns with 'BC', Postal Code,
#Deed No, Register No, Valuation Date, Capture Date 
# Stand No, Sect Tittle, Account Number, TarrifCode(Because it is dependent ), Street No,Up-Level Size,  Exemp Size,Exemp Value,  
#Main Roll No, Owner Type(dependent on), Valuation date, Up-LevelInd, usage description, Zone Description, SGNumber, 
# Status, (Postal Adrress1, 2, 3, 4, 5), BC Date, Postal Code, Deed No,(All BC Values), Register No.
#Valuation Date(date time type), Capture Date
# Usage Code: Discard later if needed
# What does Empty Stand Ind - indicate?(remove if possible)
# Investigate the meaning of Valuation Type Values
#More info on: Meter Book
# Verify if Postal Code matches with the township
# 
data_new = data.drop(columns = ['Stand No', 'Sect Title No', 'Account No', 'TariffCode',
       'Street No', 'Up-Level Size','Exemp Size',
       'Exemp Value', 'Main Roll No', 'Owner Type',
       'Valuation Date', 'Up-LevelInd','BC Create Date',
       'BC Orig Date', 'BC Altern Date', 'BC Value','BC Tariff',
       'Zone Descrip', 'UsageDescrip','SGNumber','Status',
       'Postal Addr1', 'Postal Addr2', 'Postal Addr3',
       'Postal Addr4', 'Postal Addr5', 'Postal Code',
       'Deed Date', 'Deed No', 'Register Date', 'Register No'])
data_new = data_new.drop(columns = [data_new.columns[-1]])
#Capute date is not properly named, so we must remove it using the indexs
data_new.head()

,Township,StandDescrip,Owner Name,Prop Category,Physical Addr,Size,MarketValue,Supp RollNo,Effect Date,Zone Code,Usage Code,Empty Stand Ind,Valuation Type,Ward,Cycle,Meter Book,Val Cnt
0,NELSPRUIT TOWN,GED 1 ERF 1,NELSPRUIT HARDWARE,BUS,PAUL KRUGER STREET,862.0,2900000.0,4.0,20090805.0,BUS2,BUS2,N,VI,14.0,3.0,MB0201,1.0
1,NELSPRUIT TOWN,RESTANT VAN ERF 1,MINTY IE & SONS PTY LTD,BUS,ANDERSON STREET,625.0,1700000.0,1.0,20090701.0,BUS2,BUS2,NaN,VG,14.0,3.0,MB0005,1.0
2,NELSPRUIT TOWN,ERF 2,MINTYS UNIVERSAL STORES LTD,BUS,ANDREW STREET,1487.0,6200000.0,4.0,20090805.0,BUS2,BUS2,NaN,VI,14.0,3.0,MB0201,1.0
3,NELSPRUIT TOWN,ERF 3,LOWMIN PROPERTIES (PTY) LTD,BUS,ANDERSON STREET,1487.0,7300000.0,4.0,20090805.0,BUS2,BUS2,NaN,VI,14.0,3.0,MB0005,1.0
4,NELSPRUIT TOWN,ERF 4,NELMIN ERF 4 PROPERTIES (PTY) LTD,BUS,ANDERSON STREET,1487.0,4500000.0,6.0,20100125.0,BUS2,BUS2,N,VI,14.0,3.0,MB0005,1.0


In [6]:
#Data wrangling of dataset from Column 7 to Coulumn 14
data_section = data_new.iloc[:,7:14]
data_section

,Supp RollNo,Effect Date,Zone Code,Usage Code,Empty Stand Ind,Valuation Type,Ward
0,4.0,20090805.0,BUS2,BUS2,N,VI,14.0
1,1.0,20090701.0,BUS2,BUS2,NaN,VG,14.0
2,4.0,20090805.0,BUS2,BUS2,NaN,VI,14.0
3,4.0,20090805.0,BUS2,BUS2,NaN,VI,14.0
4,6.0,20100125.0,BUS2,BUS2,N,VI,14.0
...,...,...,...,...,...,...,...
57247,1.0,20090701.0,RES2,RES2,N,VG,30.0
57248,1.0,20090701.0,RES2,RES2,N,VG,30.0
57249,1.0,20090701.0,RES2,RES2,N,VG,30.0
57250,2.0,20090702.0,RES1,RES1,N,VI,30.0


In [7]:
#There is only N and null values in 'Empty Stand Ind'. The question is 'is null equal to yes?', this assumption is far fetched thus might be incorrect, based on that we take out this column
data_new = data_new.drop(columns = 'Empty Stand Ind')
data_section = data_new.iloc[:,7:13]
data_section

,Supp RollNo,Effect Date,Zone Code,Usage Code,Valuation Type,Ward
0,4.0,20090805.0,BUS2,BUS2,VI,14.0
1,1.0,20090701.0,BUS2,BUS2,VG,14.0
2,4.0,20090805.0,BUS2,BUS2,VI,14.0
3,4.0,20090805.0,BUS2,BUS2,VI,14.0
4,6.0,20100125.0,BUS2,BUS2,VI,14.0
...,...,...,...,...,...,...
57247,1.0,20090701.0,RES2,RES2,VG,30.0
57248,1.0,20090701.0,RES2,RES2,VG,30.0
57249,1.0,20090701.0,RES2,RES2,VG,30.0
57250,2.0,20090702.0,RES1,RES1,VI,30.0


In [8]:
#Checking columns with null values
ncols_missing = data_section.isnull().any(axis=0).sum()
print(ncols_missing)
#Checking rows with null values
nrows_missing = data_section.isnull().any(axis=1).sum()
print(nrows_missing)

6
6


In [9]:
#All 6 columns have missing values
#6 rows have missing values

#For seing which rows index are null in respect to each column.
print(np.where(data_section['Supp RollNo'].isnull() == True)[0])
print("rows 30400, 40754, 40767, 40800, 40815 and 40834 are null for Supp RollNo")
print(np.where(data_section['Effect Date'].isnull() == True)[0])
print("rows 30400, 40754, 40767, 40800, 40815 and 40834 are null for Effect Date")
print(np.where(data_section['Zone Code'].isnull() == True)[0])
print("rows 30400, 40754, 40767, 40800, 40815 and 40834 are null for Zone Code")
print(np.where(data_section['Usage Code'].isnull() == True)[0])
print("rows 30400, 40754, 40767, 40800, 40815 and 40834 are null for Usage Code")
print(np.where(data_section['Valuation Type'].isnull() == True)[0])
print("rows 30400, 40754, 40767, 40800, 40815 and 40834 are null for Valuation Type")
print(np.where(data_section['Ward'].isnull() == True)[0])
print("rows 30400, 40754, 40767, 40800, 40815 and 40834 are null for Ward")

[30400 40754 40767 40800 40815 40834]
rows 30400, 40754, 40767, 40800, 40815 and 40834 are null for Supp RollNo
[30400 40754 40767 40800 40815 40834]
rows 30400, 40754, 40767, 40800, 40815 and 40834 are null for Effect Date
[30400 40754 40767 40800 40815 40834]
rows 30400, 40754, 40767, 40800, 40815 and 40834 are null for Zone Code
[30400 40754 40767 40800 40815 40834]
rows 30400, 40754, 40767, 40800, 40815 and 40834 are null for Usage Code
[30400 40754 40767 40800 40815 40834]
rows 30400, 40754, 40767, 40800, 40815 and 40834 are null for Valuation Type
[30400 40754 40767 40800 40815 40834]
rows 30400, 40754, 40767, 40800, 40815 and 40834 are null for Ward


In [10]:
print(data_new.loc[[30400, 40754, 40767, 40800, 40815, 40834]])
#All the rows with missing values are removed
data_new = data_new.drop([30400, 40754, 40767, 40800, 40815, 40834]);

            Township  StandDescrip                   Owner Name Prop Category  \
30400    ROCKY DRIFT  ERF 00000010   TEN ROCKY DRIFT PTY LTD              NaN   
40754  KANYAMAZANE-A           NaN      SM NGOMANE                        NaN   
40767  KANYAMAZANE-A           NaN      HM NGOMANE                        NaN   
40800  KANYAMAZANE-A           NaN       F NGWAMBA                        NaN   
40815  KANYAMAZANE-A           NaN  M & NR THELA/NKOSI                    NaN   
40834  KANYAMAZANE-A           NaN      MS NHLAPO                         NaN   

      Physical Addr  Size  MarketValue  Supp RollNo  Effect Date Zone Code  \
30400           NaN   NaN          NaN          NaN          NaN       NaN   
40754           NaN   NaN          NaN          NaN          NaN       NaN   
40767           NaN   NaN          NaN          NaN          NaN       NaN   
40800           NaN   NaN          NaN          NaN          NaN       NaN   
40815           NaN   NaN          NaN    

In [16]:
data_section = data_new.iloc[:,7:13]
#Checking columns with null values
ncols_missing = data_section.isnull().any(axis=0).sum()
print(ncols_missing)
#Checking rows with null values
nrows_missing = data_section.isnull().any(axis=1).sum()
print(nrows_missing)

0
0


In [17]:
#Making my dictionaries
dic = pd.Series(data_section['Zone Code'].unique())
dic.index = dic.index + 1
dic = pd.Series(dic.index.values, index = dic)

dic2 = pd.Series(data_section['Usage Code'].unique())
dic2.index = dic2.index + 1
dic2 = pd.Series(dic2.index.values, index = dic2)

dic3 = pd.Series(data_section['Valuation Type'].unique())
dic3.index = dic3.index + 1
dic3 = pd.Series(dic3.index.values, index = dic3)
#dic3

In [18]:
#Encoding based on the dictionaries
lookup_table = dict(dic)

def lookup_customer(val):
    return lookup_table.get(val, np.nan)


data_section['Zone Code'] = data_section['Zone Code'].apply(lambda x: lookup_customer(x));

lookup_table = dict(dic2)
data_section['Usage Code'] = data_section['Usage Code'].apply(lambda x: lookup_customer(x));

lookup_table = dict(dic3)
data_section['Valuation Type'] = data_section['Valuation Type'].apply(lambda x: lookup_customer(x));

data_section
#df_new.head(20)

,Supp RollNo,Effect Date,Zone Code,Usage Code,Valuation Type,Ward
0,4.0,20090805.0,1,1,1,14.0
1,1.0,20090701.0,1,1,2,14.0
2,4.0,20090805.0,1,1,1,14.0
3,4.0,20090805.0,1,1,1,14.0
4,6.0,20100125.0,1,1,1,14.0
...,...,...,...,...,...,...
57247,1.0,20090701.0,14,15,2,30.0
57248,1.0,20090701.0,14,15,2,30.0
57249,1.0,20090701.0,14,15,2,30.0
57250,2.0,20090702.0,9,9,1,30.0


In [19]:
###Supp RollNo is Supplimentary roll number
##Every property in every municipality should (hypothetically) be on a roll, but because properties are continuously coming into existence and ceasing to exist, new rolls are created (these are referred to as supplementary rolls) to include any properties that have not been previously recorded on another, prior, general roll.
data_section.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57246 entries, 0 to 57251
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Supp RollNo     57246 non-null  float64
 1   Effect Date     57246 non-null  float64
 2   Zone Code       57246 non-null  int64  
 3   Usage Code      57246 non-null  int64  
 4   Valuation Type  57246 non-null  int64  
 5   Ward            57246 non-null  float64
dtypes: float64(3), int64(3)
memory usage: 3.1 MB


In [20]:
#convert column Effect date to date time
data_section['Effect Date'] = pd.to_datetime(data_section['Effect Date'], format='%Y%m%d') 
data_section

,Supp RollNo,Effect Date,Zone Code,Usage Code,Valuation Type,Ward
0,4.0,2009-08-05,1,1,1,14.0
1,1.0,2009-07-01,1,1,2,14.0
2,4.0,2009-08-05,1,1,1,14.0
3,4.0,2009-08-05,1,1,1,14.0
4,6.0,2010-01-25,1,1,1,14.0
...,...,...,...,...,...,...
57247,1.0,2009-07-01,14,15,2,30.0
57248,1.0,2009-07-01,14,15,2,30.0
57249,1.0,2009-07-01,14,15,2,30.0
57250,2.0,2009-07-02,9,9,1,30.0


In [27]:
data_new.iloc[:,7:13] = data_section
data_new

,Township,StandDescrip,Owner Name,Prop Category,Physical Addr,Size,MarketValue,Supp RollNo,Effect Date,Zone Code,Usage Code,Valuation Type,Ward,Cycle,Meter Book,Val Cnt
0,NELSPRUIT TOWN,GED 1 ERF 1,NELSPRUIT HARDWARE,BUS,PAUL KRUGER STREET,862.0,2900000.0,4.0,2009-08-05,1,1,1,14.0,3.0,MB0201,1.0
1,NELSPRUIT TOWN,RESTANT VAN ERF 1,MINTY IE & SONS PTY LTD,BUS,ANDERSON STREET,625.0,1700000.0,1.0,2009-07-01,1,1,2,14.0,3.0,MB0005,1.0
2,NELSPRUIT TOWN,ERF 2,MINTYS UNIVERSAL STORES LTD,BUS,ANDREW STREET,1487.0,6200000.0,4.0,2009-08-05,1,1,1,14.0,3.0,MB0201,1.0
3,NELSPRUIT TOWN,ERF 3,LOWMIN PROPERTIES (PTY) LTD,BUS,ANDERSON STREET,1487.0,7300000.0,4.0,2009-08-05,1,1,1,14.0,3.0,MB0005,1.0
4,NELSPRUIT TOWN,ERF 4,NELMIN ERF 4 PROPERTIES (PTY) LTD,BUS,ANDERSON STREET,1487.0,4500000.0,6.0,2010-01-25,1,1,1,14.0,3.0,MB0005,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57247,WHITERIVER ACRES,ERF 00000056,INGWE TRUST,RES,PETER GRAHAM,91.0,475000.0,1.0,2009-07-01,14,15,2,30.0,40.0,MB4207,1.0
57248,WHITERIVER ACRES,ERF 00000056,S EIGENHUIS,RES,PETER GRAHAM,91.0,475000.0,1.0,2009-07-01,14,15,2,30.0,40.0,MB4207,1.0
57249,WHITERIVER ACRES,ERF 00000056,JS STANDER,RES,PETER GRAHAM,91.0,475000.0,1.0,2009-07-01,14,15,2,30.0,40.0,MB4207,1.0
57250,WILLIAM LYNNSTRAAT 51,PORTION 00005 OF ERF 00000332,JS STANDER,RES,WILLIAM LYNN STREET,105.0,459000.0,2.0,2009-07-02,9,9,1,30.0,40.0,MB4221,1.0


In [28]:
#Note that ERF is afrikaans and it means inherit
#Restant mean remaining when translated from afrikaans to english
#Restant van Erf therefore means remaing of land

In [29]:
df = pd.DataFrame()
df = data_new.copy()
df

,Township,StandDescrip,Owner Name,Prop Category,Physical Addr,Size,MarketValue,Supp RollNo,Effect Date,Zone Code,Usage Code,Valuation Type,Ward,Cycle,Meter Book,Val Cnt
0,NELSPRUIT TOWN,GED 1 ERF 1,NELSPRUIT HARDWARE,BUS,PAUL KRUGER STREET,862.0,2900000.0,4.0,2009-08-05,1,1,1,14.0,3.0,MB0201,1.0
1,NELSPRUIT TOWN,RESTANT VAN ERF 1,MINTY IE & SONS PTY LTD,BUS,ANDERSON STREET,625.0,1700000.0,1.0,2009-07-01,1,1,2,14.0,3.0,MB0005,1.0
2,NELSPRUIT TOWN,ERF 2,MINTYS UNIVERSAL STORES LTD,BUS,ANDREW STREET,1487.0,6200000.0,4.0,2009-08-05,1,1,1,14.0,3.0,MB0201,1.0
3,NELSPRUIT TOWN,ERF 3,LOWMIN PROPERTIES (PTY) LTD,BUS,ANDERSON STREET,1487.0,7300000.0,4.0,2009-08-05,1,1,1,14.0,3.0,MB0005,1.0
4,NELSPRUIT TOWN,ERF 4,NELMIN ERF 4 PROPERTIES (PTY) LTD,BUS,ANDERSON STREET,1487.0,4500000.0,6.0,2010-01-25,1,1,1,14.0,3.0,MB0005,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57247,WHITERIVER ACRES,ERF 00000056,INGWE TRUST,RES,PETER GRAHAM,91.0,475000.0,1.0,2009-07-01,14,15,2,30.0,40.0,MB4207,1.0
57248,WHITERIVER ACRES,ERF 00000056,S EIGENHUIS,RES,PETER GRAHAM,91.0,475000.0,1.0,2009-07-01,14,15,2,30.0,40.0,MB4207,1.0
57249,WHITERIVER ACRES,ERF 00000056,JS STANDER,RES,PETER GRAHAM,91.0,475000.0,1.0,2009-07-01,14,15,2,30.0,40.0,MB4207,1.0
57250,WILLIAM LYNNSTRAAT 51,PORTION 00005 OF ERF 00000332,JS STANDER,RES,WILLIAM LYNN STREET,105.0,459000.0,2.0,2009-07-02,9,9,1,30.0,40.0,MB4221,1.0


In [30]:
print(df['Township'].isnull().sum())               #printing null values of Township
print(df['StandDescrip'].isnull().sum())           #printing null values of StandDescrip
print(df['Owner Name'].isnull().sum())             #printing null values of Owner Name
print(df['Prop Category'].isnull().sum())          #printing null values of Prop Category
print(df['Physical Addr'].isnull().sum())          #printing null values of Physical Addr
print(df['Size'].isnull().sum())                   #printing null values of Size
print(df['MarketValue'].isnull().sum())            #printing null values of MarketValue

0
28609
13
0
279
0
0


In [31]:
#encode prop category
#1 BUS and #2 RES #3 GOV #4 MUN #5 POW #6 OTH #7 PSI #8 AG #9 RUR
df['Prop Category'] = np.where(df['Prop Category'] == 'BUS', 1 ,df['Prop Category'])
df['Prop Category'] = np.where(df['Prop Category'] == 'RES', 2 ,df['Prop Category'])
df['Prop Category'] = np.where(df['Prop Category'] == 'GOV', 3 ,df['Prop Category'])
df['Prop Category'] = np.where(df['Prop Category'] == 'MUN', 4 ,df['Prop Category'])
df['Prop Category'] = np.where(df['Prop Category'] == 'POW', 5 ,df['Prop Category'])
df['Prop Category'] = np.where(df['Prop Category'] == 'OTH', 6 ,df['Prop Category'])
df['Prop Category'] = np.where(df['Prop Category'] == 'PSI', 7 ,df['Prop Category'])
df['Prop Category'] = np.where(df['Prop Category'] == 'AG',  8 ,df['Prop Category'])
df['Prop Category'] = np.where(df['Prop Category'] == 'RUR', 9 ,df['Prop Category'])

In [32]:
#encode missing owners name with 'other'
df['Owner Name'] = df['Owner Name'].fillna('other')
df['Owner Name'].isnull().sum()

0

In [33]:
#only managed to recover 1059 StandDescrip values
def lookup_stand_desc(stand_no):
    return dictionary.get(stand_no)

df1 = data.copy()
stand_desc = df['StandDescrip'].tolist() + ['nan', 'nan', 'nan', 'nan', 'nan', 'nan']
series = pd.Series(stand_desc, index =df1['Stand No']).dropna()
dictionary = series.to_dict()

df['StandDescrip'] = df1['Stand No'].apply(lookup_stand_desc)

In [34]:
df['StandDescrip'].isnull().sum()

27550

In [35]:
#encode missing values for physical address
df['Physical Addr'] = df['Physical Addr'].fillna('Some Street')
df['Physical Addr'].isnull().sum()

0

In [36]:
physical_addr = df['Physical Addr'].tolist()
streets  = []
street_num  = 1
for i, street in enumerate(physical_addr):
    if street not in streets:
        streets.append(street)
        physical_addr[i] = streets.index(street) + 1
    else:
        physical_addr[i] = streets.index(street) + 1
df['Physical Addr'] = physical_addr

In [37]:
df['Prop Category']

0        1
1        1
2        1
3        1
4        1
        ..
57247    2
57248    2
57249    2
57250    2
57251    2
Name: Prop Category, Length: 57246, dtype: object

In [38]:
df

,Township,StandDescrip,Owner Name,Prop Category,Physical Addr,Size,MarketValue,Supp RollNo,Effect Date,Zone Code,Usage Code,Valuation Type,Ward,Cycle,Meter Book,Val Cnt
0,NELSPRUIT TOWN,GED 1 ERF 1,NELSPRUIT HARDWARE,1,1,862.0,2900000.0,4.0,2009-08-05,1,1,1,14.0,3.0,MB0201,1.0
1,NELSPRUIT TOWN,RESTANT VAN ERF 1,MINTY IE & SONS PTY LTD,1,2,625.0,1700000.0,1.0,2009-07-01,1,1,2,14.0,3.0,MB0005,1.0
2,NELSPRUIT TOWN,ERF 2,MINTYS UNIVERSAL STORES LTD,1,3,1487.0,6200000.0,4.0,2009-08-05,1,1,1,14.0,3.0,MB0201,1.0
3,NELSPRUIT TOWN,ERF 3,LOWMIN PROPERTIES (PTY) LTD,1,2,1487.0,7300000.0,4.0,2009-08-05,1,1,1,14.0,3.0,MB0005,1.0
4,NELSPRUIT TOWN,ERF 4,NELMIN ERF 4 PROPERTIES (PTY) LTD,1,2,1487.0,4500000.0,6.0,2010-01-25,1,1,1,14.0,3.0,MB0005,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57247,WHITERIVER ACRES,nan,INGWE TRUST,2,740,91.0,475000.0,1.0,2009-07-01,14,15,2,30.0,40.0,MB4207,1.0
57248,WHITERIVER ACRES,nan,S EIGENHUIS,2,740,91.0,475000.0,1.0,2009-07-01,14,15,2,30.0,40.0,MB4207,1.0
57249,WHITERIVER ACRES,nan,JS STANDER,2,740,91.0,475000.0,1.0,2009-07-01,14,15,2,30.0,40.0,MB4207,1.0
57250,WILLIAM LYNNSTRAAT 51,nan,JS STANDER,2,738,105.0,459000.0,2.0,2009-07-02,9,9,1,30.0,40.0,MB4221,1.0
